In [1]:
import sys
import os
import shutil
import datetime
import re
import pandas as pd
from collections import Counter
#sys.path.append(os.path.abspath("/home/jose/Dropbox/Doktorarbeit/"))
#from reading_robot import load_data, tei2text, text2features, classify, process_labels, describe_data,feature_analysis, cull_data, st_tests, metadata2numbers, cluster, call_stylo, prepare_subcorpus
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
%matplotlib inline
from shutil import copyfile
import numpy as np
import glob

# Read Data

In [2]:
wdir = "C:/Users/calvotello/Dropbox/MTB/investigacion/estudio_parametros/data/disco/original_corpus_author/"

In [3]:
files = glob.glob(wdir + "/*.txt")


In [15]:
files

['C:/Users/calvotello/Dropbox/MTB/investigacion/estudio_parametros/data/disco/original_corpus_author\\Agustini-Delmira_1.txt',
 'C:/Users/calvotello/Dropbox/MTB/investigacion/estudio_parametros/data/disco/original_corpus_author\\Agustini-Delmira_2.txt',
 'C:/Users/calvotello/Dropbox/MTB/investigacion/estudio_parametros/data/disco/original_corpus_author\\Alarcón-Pedro-Antonio-de_1.txt',
 'C:/Users/calvotello/Dropbox/MTB/investigacion/estudio_parametros/data/disco/original_corpus_author\\Alarcón-Pedro-Antonio-de_2.txt',
 'C:/Users/calvotello/Dropbox/MTB/investigacion/estudio_parametros/data/disco/original_corpus_author\\Althaus-Clemente_1.txt',
 'C:/Users/calvotello/Dropbox/MTB/investigacion/estudio_parametros/data/disco/original_corpus_author\\Althaus-Clemente_2.txt',
 'C:/Users/calvotello/Dropbox/MTB/investigacion/estudio_parametros/data/disco/original_corpus_author\\Arnao-Nicolás_1.txt',
 'C:/Users/calvotello/Dropbox/MTB/investigacion/estudio_parametros/data/disco/original_corpus_auth

In [16]:
files_df = pd.DataFrame(files, columns=["path"])

In [17]:
files_df

,path
0,C:/Users/calvotello/Dropbox/MTB/investigacion/...
1,C:/Users/calvotello/Dropbox/MTB/investigacion/...
2,C:/Users/calvotello/Dropbox/MTB/investigacion/...
3,C:/Users/calvotello/Dropbox/MTB/investigacion/...
4,C:/Users/calvotello/Dropbox/MTB/investigacion/...
...,...
59,C:/Users/calvotello/Dropbox/MTB/investigacion/...
60,C:/Users/calvotello/Dropbox/MTB/investigacion/...
61,C:/Users/calvotello/Dropbox/MTB/investigacion/...
62,C:/Users/calvotello/Dropbox/MTB/investigacion/...


In [18]:
files_df["file_name"] = files_df["path"].str.findall("\\\(.*?)\.txt").str[0]

In [19]:
files_df

,path,file_name
0,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Agustini-Delmira_1
1,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Agustini-Delmira_2
2,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Alarcón-Pedro-Antonio-de_1
3,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Alarcón-Pedro-Antonio-de_2
4,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Althaus-Clemente_1
...,...,...
59,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Urbach-Carlos-Póo_2
60,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Valverde-López-Carlos_1
61,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Valverde-López-Carlos_2
62,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Velilla-Rodríguez-Mercedes-de_1


In [20]:
files_df["author"] = files_df["file_name"].str.findall("^(.*?)_").str[0]

In [21]:
files_df["text_title"] = files_df["file_name"].str.findall("_(.*?)$").str[0]

In [22]:
files_df

,path,file_name,author,text_title
0,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Agustini-Delmira_1,Agustini-Delmira,1
1,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Agustini-Delmira_2,Agustini-Delmira,2
2,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Alarcón-Pedro-Antonio-de_1,Alarcón-Pedro-Antonio-de,1
3,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Alarcón-Pedro-Antonio-de_2,Alarcón-Pedro-Antonio-de,2
4,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Althaus-Clemente_1,Althaus-Clemente,1
...,...,...,...,...
59,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Urbach-Carlos-Póo_2,Urbach-Carlos-Póo,2
60,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Valverde-López-Carlos_1,Valverde-López-Carlos,1
61,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Valverde-López-Carlos_2,Valverde-López-Carlos,2
62,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Velilla-Rodríguez-Mercedes-de_1,Velilla-Rodríguez-Mercedes-de,1


In [23]:
sorted(list(files_df.author.unique()))


['Agustini-Delmira',
 'Alarcón-Pedro-Antonio-de',
 'Althaus-Clemente',
 'Arnao-Nicolás',
 'Balart-Federico',
 'Balmori-Jesús',
 'Baralt-Rafael-María',
 'Bernabé-Manuel',
 'Casal-Julián-del',
 'Flores-Roa-Julio',
 'Gavira-José-Hernández',
 'Herrera-y-Reissig-Julio',
 'Hilario-Zoilo-J.',
 'Labaila-Jacinto',
 'Lles-Francisco',
 'López-de-Ayala-Adelardo',
 'Marfori-Isidro',
 'Matamoros-Mercedes',
 'Monte-Ricardo-del',
 'Navarro-Montes-de-Oca-José',
 'Núñez-de-Arce-Gaspar',
 'Ortiz-Carlos',
 'Othón-Manuel-José',
 'Palacio-Manuel-del',
 'Ros-de-Olano-Antonio',
 'Rubalcava-Manuel-Justo-de',
 'Santos-Chocano-José',
 'Selgas-y-Carrasco-José',
 'Ugarte-y-Pages-Francisco-Javier',
 'Urbach-Carlos-Póo',
 'Valverde-López-Carlos',
 'Velilla-Rodríguez-Mercedes-de']

In [24]:
for index, row in files_df.iterrows():
    with open(row["path"], "r", errors="replace", encoding="utf-8") as fin:
        content_str = fin.read()
    files_df.loc[index, "text"] = content_str

In [25]:
files_df

,path,file_name,author,text_title,text
0,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Agustini-Delmira_1,Agustini-Delmira,1,\n\n\n\n¿Te acuerdas? El arroyo fue la serpien...
1,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Agustini-Delmira_2,Agustini-Delmira,2,\n\n\n\nÍbamos en la tarde que caía\nrápidamen...
2,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Alarcón-Pedro-Antonio-de_1,Alarcón-Pedro-Antonio-de,1,"\n\n\n\nFumaba yo, tendido en mi butaca,\ncuan..."
3,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Alarcón-Pedro-Antonio-de_2,Alarcón-Pedro-Antonio-de,2,"\n\n\n\nSolo, negado, escarnecido, muerto,\nen..."
4,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Althaus-Clemente_1,Althaus-Clemente,1,"\n\n\n\nUn día, España, en tu anchuroso imperi..."
...,...,...,...,...,...
59,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Urbach-Carlos-Póo_2,Urbach-Carlos-Póo,2,\n\n\n\nSomos nosotros pálido pintores\nque di...
60,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Valverde-López-Carlos_1,Valverde-López-Carlos,1,"\n\n\n\nSin Dios, porque lo olvida en su locur..."
61,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Valverde-López-Carlos_2,Valverde-López-Carlos,2,\n\n\n\nSi al persistir en ese afán terrible\n...
62,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Velilla-Rodríguez-Mercedes-de_1,Velilla-Rodríguez-Mercedes-de,1,"\n\n\n\nAlma sublime, de virtud modelo,\nvida ..."


In [26]:
files_df.index = files_df.file_name

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
reg_ex = r"(?u)\b[\w_@]+\b|[¶\(»\]\?\.\–\!’•\|“\>\)\-\—\:\}\*\&…¿\/=¡\"\'·+\{\#\[;­,«~]"
sort_columns_by_frequency = True
lowercase = True
max_features = 1000000
analyzer = "word" #{‘word’, ‘char’, ‘char_wb’}
ngram_range = (1,1) #ngram_range = None

#vectorizer = CountVectorizer()
#X = vectorizer.fit_transform(corpus)

#vectorizer = CountVectorizer()

vec = CountVectorizer(
        token_pattern = reg_ex,
        lowercase = lowercase,
        max_features = max_features,
        analyzer = analyzer,
        ngram_range = ngram_range,
        )

tokens = vec.fit_transform(files_df.text.tolist())

tokens_df = pd.DataFrame(tokens.toarray(), columns = vec.get_feature_names(), index = files_df["file_name"])

In [29]:
tokens_df

,!,"""",',(,),",",-,.,/,101,...,únicamente,único,únicos,úrsula,útil,útilísimo,übn,–,—,…
file_name,,,,,,,,,,,,,,,,,,,,,
Agustini-Delmira_1,35,0,0,0,0,129,13,20,0,0,...,0,0,0,0,0,0,0,0,0,0
Agustini-Delmira_2,18,0,1,0,0,106,9,18,0,0,...,0,0,0,0,0,0,0,0,0,0
Alarcón-Pedro-Antonio-de_1,23,0,0,0,0,138,0,11,0,0,...,0,1,0,0,0,0,0,0,0,0
Alarcón-Pedro-Antonio-de_2,23,0,0,2,2,115,0,19,0,0,...,0,0,0,0,0,0,0,0,0,0
Althaus-Clemente_1,23,0,0,0,0,226,0,24,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Urbach-Carlos-Póo_2,6,0,1,0,0,84,0,10,0,0,...,0,0,0,0,0,0,0,0,0,0
Valverde-López-Carlos_1,1,0,0,0,0,55,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
Valverde-López-Carlos_2,4,0,0,0,0,85,0,32,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
files_df["tokens_sum"] = tokens_df.sum(axis=1)

In [31]:
files_df

,path,file_name,author,text_title,text,tokens_sum
file_name,,,,,,
Agustini-Delmira_1,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Agustini-Delmira_1,Agustini-Delmira,1,\n\n\n\n¿Te acuerdas? El arroyo fue la serpien...,1927
Agustini-Delmira_2,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Agustini-Delmira_2,Agustini-Delmira,2,\n\n\n\nÍbamos en la tarde que caía\nrápidamen...,1773
Alarcón-Pedro-Antonio-de_1,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Alarcón-Pedro-Antonio-de_1,Alarcón-Pedro-Antonio-de,1,"\n\n\n\nFumaba yo, tendido en mi butaca,\ncuan...",1397
Alarcón-Pedro-Antonio-de_2,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Alarcón-Pedro-Antonio-de_2,Alarcón-Pedro-Antonio-de,2,"\n\n\n\nSolo, negado, escarnecido, muerto,\nen...",1428
Althaus-Clemente_1,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Althaus-Clemente_1,Althaus-Clemente,1,"\n\n\n\nUn día, España, en tu anchuroso imperi...",2740
...,...,...,...,...,...,...
Urbach-Carlos-Póo_2,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Urbach-Carlos-Póo_2,Urbach-Carlos-Póo,2,\n\n\n\nSomos nosotros pálido pintores\nque di...,1241
Valverde-López-Carlos_1,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Valverde-López-Carlos_1,Valverde-López-Carlos,1,"\n\n\n\nSin Dios, porque lo olvida en su locur...",1222
Valverde-López-Carlos_2,C:/Users/calvotello/Dropbox/MTB/investigacion/...,Valverde-López-Carlos_2,Valverde-López-Carlos,2,\n\n\n\nSi al persistir en ese afán terrible\n...,1294


In [32]:
files_df.author.value_counts().mean()

2.0

In [33]:
files_df.shape[0]

64

In [34]:
files_df.author.value_counts()

Ros-de-Olano-Antonio               2
Baralt-Rafael-María                2
Urbach-Carlos-Póo                  2
Hilario-Zoilo-J.                   2
Rubalcava-Manuel-Justo-de          2
Valverde-López-Carlos              2
Alarcón-Pedro-Antonio-de           2
Labaila-Jacinto                    2
Santos-Chocano-José                2
Balmori-Jesús                      2
Flores-Roa-Julio                   2
Herrera-y-Reissig-Julio            2
Monte-Ricardo-del                  2
Balart-Federico                    2
Agustini-Delmira                   2
Matamoros-Mercedes                 2
Arnao-Nicolás                      2
Ortiz-Carlos                       2
Velilla-Rodríguez-Mercedes-de      2
Ugarte-y-Pages-Francisco-Javier    2
Navarro-Montes-de-Oca-José         2
Lles-Francisco                     2
Gavira-José-Hernández              2
Bernabé-Manuel                     2
Palacio-Manuel-del                 2
López-de-Ayala-Adelardo            2
Casal-Julián-del                   2
N

In [36]:
files_df.to_csv("./../data/disco/metadata.tsv", sep="\t")

In [37]:
number_texts = files_df.shape[0]
number_authors = len(files_df.author.unique())
mean_number_text_author = files_df.author.value_counts().mean()
std_number_text_author = files_df.author.value_counts().std()
mean_length_text = files_df.tokens_sum.mean()
std_length_text = files_df.tokens_sum.std()
distribution_authors_texts = files_df.author.value_counts()


In [41]:
summary_corpus_df = pd.DataFrame(
            [number_texts,   number_authors,  mean_number_text_author,   std_number_text_author,  mean_length_text, std_length_text, distribution_authors_texts  ],
    index=["number_texts","number_authors","mean_number_text_author", "std_number_text_author","mean_length","std_length", "distribution_authors_texts"  ],
    columns = ["19-20-poesia"]).T

In [42]:
summary_corpus_df

,number_texts,number_authors,mean_number_text_author,std_number_text_author,mean_length,std_length,distribution_authors_texts
19-20-poesia,64,32,2,0,1837.72,689.4,Ros-de-Olano-Antonio 2 Baralt-Ra...


In [43]:
summary_corpus_df.to_csv("./../data/disco/summary.tsv", sep="\t")